# L4: Tools for a Customer Outreach Campaign

In this lesson, you will learn more about Tools. You'll focus on three key elements of Tools:
- Versatility
- Fault Tolerance
- Caching

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM
- [Serper](https://serper.dev)

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key, pretty_print_result
from utils import get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## Creating Agents

In [4]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True
)

In [5]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True
)

## Creating Tools

### crewAI Tools

In [6]:
from crewai_tools import DirectoryReadTool, \
                         FileReadTool, \
                         SerperDevTool

In [7]:
directory_read_tool = DirectoryReadTool(directory='./instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

### Custom Tool
- Create a custom tool using crewAi's [BaseTool](https://docs.crewai.com/core-concepts/Tools/#subclassing-basetool) class

In [8]:
from crewai_tools import BaseTool

- Every Tool needs to have a `name` and a `description`.
- For simplicity and classroom purposes, `SentimentAnalysisTool` will return `positive` for every text.
- When running locally, you can customize the code with your logic in the `_run` function.

In [9]:
class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")
    
    def _run(self, text: str) -> str:
        # Your custom code tool goes here
        return "positive"

In [10]:
sentiment_analysis_tool = SentimentAnalysisTool()

## Creating Tasks

- The Lead Profiling Task is using crewAI Tools.

In [11]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

- The Personalized Outreach Task is using your custom Tool `SentimentAnalysisTool`, as well as crewAI's `SerperDevTool` (search_tool).

In [12]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

## Creating the Crew

In [13]:
crew = Crew(
    agents=[sales_rep_agent, 
            lead_sales_rep_agent],
    
    tasks=[lead_profiling_task, 
           personalized_outreach_task],
	
    verbose=2,
	memory=True
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [14]:
inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch"
}

result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Sales Representative
 [INFO]: == Starting Task: Conduct an in-depth analysis of DeepLearningAI, a company in the Online Learning Platform sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


> Entering new CrewAgentExecutor chain...
I need to gather all available information on DeepLearningAI to create a comprehensive report for our sales team. It's crucial to identify key decision-makers, recent business developments, and potential needs that align with our solutions.

Action: Read a file's content
Action Input: {"file_path": "./instructions/DeepLearningAI_company_profile.txt"} 

I encountered an error while trying to use the tool. 



> Entering new CrewAgentExecutor chain...
Crafting personalized email drafts tailored to DeepLearningAI and Andrew Ng requires a deep understanding of their business, recent achievements, and future goals. By leveraging the insights gathered from the lead profiling report, I can create compelling narratives that resonate with DeepLearningAI's corporate identity.

Action:
Search the internet

Action Input:
{"search_query": "Recent achievements and future goals of DeepLearningAI and Andrew Ng"}
 


Search results: Title: AI is in business — I caught up with Andrew Ng
Link: https://www.exponentialview.co/p/andrew-ng-ai-implementation
Snippet: Andrew's accomplishments are many. He is the founder of DeepLearning.ai, founder & CEO of Landing AI, GP at AI Fund, chairman and co-founder of ...
---
Title: The Rise of AI Education: Andrew Ng & Deeplearning.ai - Seamgen
Link: https://www.seamgen.com/blog/ai-education-andrew-ng-deep-learning
Snippet: Explore Andrew Ng's AI education revolution at

- Display the final result as Markdown.

In [15]:
from IPython.display import Markdown
Markdown(result)

Email Draft to Andrew Ng, CEO of DeepLearningAI:

Subject: Empowering AI Education through Strategic Collaboration

Dear Andrew Ng,

I hope this message finds you well. I wanted to reach out to express my admiration for the incredible work you have been doing at DeepLearningAI and your continuous dedication to advancing AI education globally.

As the founder of DeepLearningAI, Landing AI, and Co-Chairman of Coursera, your influence and expertise in the field of artificial intelligence are truly commendable. Your recent appointment to Amazon's Board of Directors further solidifies your position as a thought leader in the industry.

At CrewAI, we have been following DeepLearningAI's journey closely and are inspired by your commitment to providing world-class education and hands-on training in AI. Our team specializes in AI-driven sales solutions and we believe that our personalized engagement strategies can complement DeepLearningAI's mission of empowering individuals with the skills needed to excel in AI.

We understand the importance of innovative learning methods and staying up-to-date with the latest AI developments, which is why we are excited about the opportunity to collaborate with DeepLearningAI. By aligning our expertise in sales solutions with your commitment to high-quality resources and support, we believe that together, we can make a significant impact on the AI education space.

I would love to discuss how our solutions can support DeepLearningAI's goals and contribute to the advancement of AI education. Please let me know a convenient time for a meeting or call to explore this partnership further.

Thank you for your time and consideration. I look forward to the possibility of working together to empower individuals in the field of AI.

Warm regards,

[Your Name]
Lead Sales Representative
CrewAI

---
Email Draft to Andrew Ng, CEO of DeepLearningAI:

Subject: Harnessing the Power of AI for Future Success

Dear Andrew Ng,

I trust this message finds you well. I am writing to you with great admiration for the remarkable achievements you have accomplished at DeepLearningAI and your unwavering dedication to advancing AI education.

As the founder of DeepLearningAI and CEO of Landing AI, your impact on the AI industry is truly inspiring. Your recent appointment to Amazon's Board of Directors is a testament to your expertise and influence in the field.

At CrewAI, we are impressed by DeepLearningAI's collaborative community of peers and mentors, aimed at empowering individuals with the skills needed to excel in AI. Our team specializes in AI-driven sales solutions and we believe that our offerings can further support DeepLearningAI's mission of providing high-quality resources and support to learners worldwide.

We recognize the importance of staying up-to-date with the latest AI developments and are excited about the opportunity to collaborate with DeepLearningAI. By leveraging our expertise in personalized engagement strategies, we aim to align our solutions with your goals and contribute to the growth of AI education.

I would welcome the opportunity to discuss how our solutions can add value to DeepLearningAI and support your vision for the future. Please feel free to suggest a time for a meeting or call to explore this potential partnership further.

Thank you for considering this partnership opportunity. I am excited about the prospect of working together to empower individuals in the AI field.

Best regards,

[Your Name]
Lead Sales Representative
CrewAI